In [1]:
from collections import defaultdict, deque

class MaxFlow:
    def __init__(self, vertices):
        self.graph = defaultdict(dict)
        self.vertices = vertices
        
    def add_edge(self, u, v, capacity):
        """Add an edge with capacity from u to v"""
        self.graph[u][v] = capacity
        if v not in self.graph:
            self.graph[v] = {}
    
    def bfs(self, source, sink, parent):
        """Use BFS to find if there's a path from source to sink"""
        visited = set([source])
        queue = deque([source])
        
        while queue:
            u = queue.popleft()
            
            for v in self.graph[u]:
                if v not in visited and self.graph[u][v] > 0:
                    visited.add(v)
                    queue.append(v)
                    parent[v] = u
                    if v == sink:
                        return True
        return False
    
    def ford_fulkerson(self, source, sink):
        """
        Edmonds-Karp algorithm (Ford-Fulkerson with BFS)
        Returns: (max_flow, number_of_iterations, paths_used)
        """
        parent = {}
        max_flow = 0
        iterations = 0
        paths_used = []
        
        # Create residual graph (copy of original capacities)
        residual_graph = defaultdict(dict)
        for u in self.graph:
            for v in self.graph[u]:
                residual_graph[u][v] = self.graph[u][v]
                if u not in residual_graph[v]:
                    residual_graph[v][u] = 0
        
        self.graph = residual_graph
        
        # Find augmenting paths
        while self.bfs(source, sink, parent):
            iterations += 1
            
            # Find minimum capacity along the path
            path_flow = float('inf')
            s = sink
            path = []
            
            while s != source:
                path.append(s)
                path_flow = min(path_flow, self.graph[parent[s]][s])
                s = parent[s]
            path.append(source)
            path.reverse()
            
            paths_used.append((path, path_flow))
            
            # Update residual capacities
            v = sink
            while v != source:
                u = parent[v]
                self.graph[u][v] -= path_flow
                self.graph[v][u] += path_flow
                v = parent[v]
            
            max_flow += path_flow
            parent = {}
        
        return max_flow, iterations, paths_used

flow_network = MaxFlow(5)

# Let's label vertices: 0=Source(top), 1=A, 2=B, 3=Middle, 4=Sink(bottom)
flow_network.add_edge(0, 1, 6)  # Source to A
flow_network.add_edge(0, 3, 1)  # Source to Middle
flow_network.add_edge(0, 2, 4)  # Source to B
flow_network.add_edge(1, 4, 4)  # A to Sink
flow_network.add_edge(3, 4, 1)  # Middle to Sink
flow_network.add_edge(2, 4, 6)  # B to Sink

max_flow, iterations, paths = flow_network.ford_fulkerson(0, 4)

print(f"Maximum Flow: {max_flow}")
print(f"Number of Iterations: {iterations}")
print(f"\nAugmenting paths found:")
for i, (path, flow) in enumerate(paths, 1):
    path_str = " → ".join(map(str, path))
    print(f"  Iteration {i}: {path_str} (flow: {flow})")

Maximum Flow: 9
Number of Iterations: 3

Augmenting paths found:
  Iteration 1: 0 → 1 → 4 (flow: 4)
  Iteration 2: 0 → 3 → 4 (flow: 1)
  Iteration 3: 0 → 2 → 4 (flow: 4)
